<a href="https://colab.research.google.com/github/VitalyGladyshev/med_work/blob/main/gene_01_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
try:
    import google.colab
    # Running on Google Colab, so install Biopython first
    !pip install biopython
except ImportError:
    pass


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 9.6 MB/s eta 0:00:00


In [12]:
from Bio import Entrez
from Bio import SeqIO
Entrez.email = "A.N.Other@example.com"
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET

## Извлечение информации о генах

In [64]:
handle = Entrez.efetch(db="nucleotide", id="AY851612", rettype="gb", retmode="text")
print(handle.readline().strip())
# LOCUS       AY851612                 892 bp    DNA     linear   PLN 10-APR-2007
handle.close()

LOCUS       AY851612                 892 bp    DNA     linear   PLN 10-APR-2007


In [18]:
handle = Entrez.efetch(db="gene", id="52", rettype = 'xml')
gene_locus = ET.parse(handle).getroot().find("Entrezgene/Entrezgene_locus")
region = gene_locus.find("Gene-commentary/Gene-commentary_seqs/Seq-loc/Seq-loc_int/Seq-interval")
start = int(region.find("Seq-interval_from").text) + 1
end = int(region.find("Seq-interval_to").text) + 1
gi_id = region.find("Seq-interval_id/Seq-id/Seq-id_gi").text
strand = region.find("Seq-interval_strand/Na-strand").get("value")
print(start, end, gi_id, strand)

264947 278283 568815596 plus


In [ ]:
#Выгрузить с NCBI нуклеотидную последовательность X хромосомы, для этого узнать ID хромосомы - NC_000023.11
handle = Entrez.efetch(db="nucleotide",
                       id="NC_000023.11",
                       rettype = 'fasta',
                       retmode = 'text')
#загружаем
chromosome = SeqIO.read(handle, 'fasta')

In [ ]:
#а та ли это хромосома?
chromosome.description

'NC_000023.11 Homo sapiens chromosome X, GRCh38.p14 Primary Assembly'

In [ ]:
#вырезаем из хромосомы ген, для этого узнаем локацию нужного гена. Для ACE2 это 15518197..15607211
geneACE2 = chromosome.seq[15518197:15607211]
print(geneACE2)
len(geneACE2)

...


89014

## Загрузка базы

In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [6]:
dis = pd.read_table('/gdrive/MyDrive/data/curated_gene_disease_associations.tsv')
dis

,geneId,geneSymbol,DSI,DPI,diseaseId,diseaseName,diseaseType,diseaseClass,diseaseSemanticType,score,EI,YearInitial,YearFinal,NofPmids,NofSnps,source
0,1,A1BG,0.857,0.172,C0019209,Hepatomegaly,phenotype,C06;C23,Finding,0.3,NaN,2017.0,2017.0,1,0,CTD_human
1,1,A1BG,0.857,0.172,C0036341,Schizophrenia,disease,F03,Mental or Behavioral Dysfunction,0.3,NaN,2015.0,2015.0,1,0,CTD_human
2,2,A2M,0.564,0.724,C0002395,Alzheimer's Disease,disease,C10;F03,Disease or Syndrome,0.4,0.848485,1998.0,2016.0,3,0,CTD_human
3,2,A2M,0.564,0.724,C0007102,Malignant tumor of colon,disease,C04;C06,Neoplastic Process,0.3,NaN,2004.0,2004.0,1,0,CTD_human
4,2,A2M,0.564,0.724,C0009375,Colonic Neoplasms,group,C04;C06,Neoplastic Process,0.3,NaN,2004.0,2004.0,1,0,CTD_human
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81741,109580095,HBB-LCR,0.815,0.069,C0002875,Cooley's anemia,disease,C15;C16,Disease or Syndrome,0.3,NaN,NaN,NaN,0,0,CTD_human
81742,109580095,HBB-LCR,0.815,0.069,C0005283,beta Thalassemia,disease,C15;C16,Disease or Syndrome,0.3,NaN,NaN,NaN,0,0,CTD_human
81743,109580095,HBB-LCR,0.815,0.069,C0019025,Hemoglobin F Disease,disease,C15;C16,Disease or Syndrome,0.3,NaN,NaN,NaN,0,0,CTD_human
81744,109580095,HBB-LCR,0.815,0.069,C0085578,Thalassemia Minor,disease,C15;C16,Disease or Syndrome,0.3,NaN,NaN,NaN,0,0,CTD_human


In [7]:
schis_db = dis.loc[dis['diseaseName'] == 'Schizophrenia']
schis_db

,geneId,geneSymbol,DSI,DPI,diseaseId,diseaseName,diseaseType,diseaseClass,diseaseSemanticType,score,EI,YearInitial,YearFinal,NofPmids,NofSnps,source
1,1,A1BG,0.857,0.172,C0036341,Schizophrenia,disease,F03,Mental or Behavioral Dysfunction,0.30,NaN,2015.0,2015.0,1,0,CTD_human
180,19,ABCA1,0.484,0.793,C0036341,Schizophrenia,disease,F03,Mental or Behavioral Dysfunction,0.31,1.0,2014.0,2014.0,1,0,PSYGENET
397,43,ACHE,0.494,0.793,C0036341,Schizophrenia,disease,F03,Mental or Behavioral Dysfunction,0.30,1.0,2015.0,2015.0,1,0,PSYGENET
505,52,ACP1,0.582,0.655,C0036341,Schizophrenia,disease,F03,Mental or Behavioral Dysfunction,0.32,1.0,1990.0,2018.0,1,0,PSYGENET
598,60,ACTB,0.453,0.862,C0036341,Schizophrenia,disease,F03,Mental or Behavioral Dysfunction,0.31,1.0,2012.0,2012.0,1,0,PSYGENET
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81506,100302690,DLG2-AS1,1.000,0.034,C0036341,Schizophrenia,disease,F03,Mental or Behavioral Dysfunction,0.31,1.0,2003.0,2003.0,1,0,PSYGENET
81509,100303453,TSNAX-DISC1,0.815,0.069,C0036341,Schizophrenia,disease,F03,Mental or Behavioral Dysfunction,0.31,1.0,2010.0,2013.0,2,0,PSYGENET
81661,100616452,MIR2682,0.928,0.034,C0036341,Schizophrenia,disease,F03,Mental or Behavioral Dysfunction,0.31,1.0,2015.0,2015.0,1,1,PSYGENET
81663,100616496,ERVW-4,0.676,0.517,C0036341,Schizophrenia,disease,F03,Mental or Behavioral Dysfunction,0.32,1.0,2001.0,2006.0,2,0,PSYGENET


In [14]:
dis_name_arr = dis['diseaseName'].unique()
dis_name_arr

array(['Hepatomegaly', 'Schizophrenia', "Alzheimer's Disease", ...,
       'SPINOCEREBELLAR ATAXIA 32', 'DYSTONIA 21',
       'North Carolina macular dystrophy'], dtype=object)

In [15]:
dis['diseaseName'].nunique()

10370

In [34]:
des_sample_list = np.random.choice(dis_name_arr, 2)
des_sample_list

array(['Sensory hearing loss',
       'Congenital dyserythropoietic anemia type IV'], dtype=object)

In [35]:
dis_0 = dis.loc[dis['diseaseName'] == des_sample_list[0]]
dis_0

,geneId,geneSymbol,DSI,DPI,diseaseId,diseaseName,diseaseType,diseaseClass,diseaseSemanticType,score,EI,YearInitial,YearFinal,NofPmids,NofSnps,source
617,60,ACTB,0.453,0.862,C1691779,Sensory hearing loss,disease,C09;C10;C23,Disease or Syndrome,0.3,NaN,2006.0,2006.0,1,0,CTD_human
10498,1280,COL2A1,0.474,0.724,C1691779,Sensory hearing loss,disease,C09;C10;C23,Disease or Syndrome,0.3,NaN,2006.0,2006.0,1,0,CTD_human
10768,1297,COL9A1,0.596,0.517,C1691779,Sensory hearing loss,disease,C09;C10;C23,Disease or Syndrome,0.3,NaN,2006.0,2006.0,1,0,CTD_human
10841,1302,COL11A2,0.478,0.690,C1691779,Sensory hearing loss,disease,C09;C10;C23,Disease or Syndrome,0.3,NaN,2006.0,2006.0,1,0,CTD_human
18293,2248,FGF3,0.554,0.690,C1691779,Sensory hearing loss,disease,C09;C10;C23,Disease or Syndrome,0.3,NaN,2008.0,2008.0,1,0,CTD_human
21024,2706,GJB2,0.484,0.690,C1691779,Sensory hearing loss,disease,C09;C10;C23,Disease or Syndrome,0.3,NaN,2004.0,2004.0,1,0,CTD_human
21050,2707,GJB3,0.630,0.310,C1691779,Sensory hearing loss,disease,C09;C10;C23,Disease or Syndrome,0.3,NaN,2001.0,2001.0,1,0,CTD_human
31536,4036,LRP2,0.552,0.690,C1691779,Sensory hearing loss,disease,C09;C10;C23,Disease or Syndrome,0.3,NaN,2007.0,2007.0,1,0,CTD_human
38910,5172,SLC26A4,0.554,0.759,C1691779,Sensory hearing loss,disease,C09;C10;C23,Disease or Syndrome,0.3,NaN,2005.0,2007.0,3,0,CTD_human
57452,8772,FADD,0.573,0.759,C1691779,Sensory hearing loss,disease,C09;C10;C23,Disease or Syndrome,0.3,NaN,2008.0,2008.0,1,0,CTD_human


In [37]:
dis_0['geneId']

617         60
10498     1280
10768     1297
10841     1302
18293     2248
21024     2706
21050     2707
31536     4036
38910     5172
57452     8772
61244    10265
Name: geneId, dtype: int64

In [43]:
gen_dis_0 = dis_0.iloc[0, 0]
str(gen_dis_0)

'60'

In [45]:
handle = Entrez.efetch(db="gene", id=int(gen_dis_0), rettype = 'xml')
gene_locus = ET.parse(handle).getroot().find("Entrezgene/Entrezgene_locus")
region = gene_locus.find("Gene-commentary/Gene-commentary_seqs/Seq-loc/Seq-loc_int/Seq-interval")
start = int(region.find("Seq-interval_from").text) + 1
end = int(region.find("Seq-interval_to").text) + 1
gi_id = region.find("Seq-interval_id/Seq-id/Seq-id_gi").text
strand = region.find("Seq-interval_strand/Na-strand").get("value")
print(start, end, gi_id, strand)

5527148 5530601 568815591 minus


In [46]:
handle = Entrez.efetch(db="gene", id=str(gen_dis_0), rettype = 'xml')
gene_locus = ET.parse(handle).getroot().find("Entrezgene/Entrezgene_locus")
region = gene_locus.find("Gene-commentary/Gene-commentary_seqs/Seq-loc/Seq-loc_int/Seq-interval")
start = int(region.find("Seq-interval_from").text) + 1
end = int(region.find("Seq-interval_to").text) + 1
gi_id = region.find("Seq-interval_id/Seq-id/Seq-id_gi").text
strand = region.find("Seq-interval_strand/Na-strand").get("value")
print(start, end, gi_id, strand)

5527148 5530601 568815591 minus


In [74]:
handle = Entrez.efetch(db="gene", id=int(gen_dis_0), rettype="gb", retmode="text")
myline = handle.readline()
while myline:
    print(myline)
    myline = handle.readline()
handle.close()



1. ACTB

Official Symbol: ACTB and Name: actin beta [Homo sapiens (human)]

Other Aliases: BKRNS, BNS, BRWS1, CSMH, DDS1, PS1TP5BP1, THC8

Other Designations: actin, cytoplasmic 1; I(2)-actin; PS1TP5-binding protein 1; beta cytoskeletal actin

Chromosome: 7; Location: 7p22.1

Annotation: Chromosome 7 NC_000007.14 (5527148..5530601, complement)

MIM: 102630

ID: 60





In [48]:
gen_dis_l = dis_0.iloc[-1, 0]
str(gen_dis_l)

'10265'

In [49]:
handle = Entrez.efetch(db="gene", id=str(gen_dis_l), rettype = 'xml')
gene_locus = ET.parse(handle).getroot().find("Entrezgene/Entrezgene_locus")
region = gene_locus.find("Gene-commentary/Gene-commentary_seqs/Seq-loc/Seq-loc_int/Seq-interval")
start = int(region.find("Seq-interval_from").text) + 1
end = int(region.find("Seq-interval_to").text) + 1
gi_id = region.find("Seq-interval_id/Seq-id/Seq-id_gi").text
strand = region.find("Seq-interval_strand/Na-strand").get("value")
print(start, end, gi_id, strand)

54930865 54934485 568815582 plus


In [75]:
handle = Entrez.efetch(db="gene", id=str(gen_dis_l), rettype="gb", retmode="text")
myline = handle.readline()
while myline:
    print(myline)
    myline = handle.readline()
handle.close()



1. IRX5

Official Symbol: IRX5 and Name: iroquois homeobox 5 [Homo sapiens (human)]

Other Aliases: HMMS, IRX-2a, IRXB2

Other Designations: iroquois-class homeodomain protein IRX-5; homeodomain protein IRX-2A; homeodomain protein IRXB2

Chromosome: 16; Location: 16q12.2

Annotation: Chromosome 16 NC_000016.10 (54930865..54934485)

MIM: 606195

ID: 10265





In [36]:
dis_1 = dis.loc[dis['diseaseName'] == des_sample_list[1]]
dis_1

,geneId,geneSymbol,DSI,DPI,diseaseId,diseaseName,diseaseType,diseaseClass,diseaseSemanticType,score,EI,YearInitial,YearFinal,NofPmids,NofSnps,source
62288,10661,KLF1,0.63,0.276,C3150926,Congenital dyserythropoietic anemia type IV,disease,NaN,Disease or Syndrome,0.61,1.0,1991.0,2016.0,2,3,ORPHANET;UNIPROT


In [51]:
dis_1_gen_0 = dis_1.iloc[0, 0]
str(dis_1_gen_0)

'10661'

In [52]:
handle = Entrez.efetch(db="gene", id=int(dis_1_gen_0), rettype = 'xml')
gene_locus = ET.parse(handle).getroot().find("Entrezgene/Entrezgene_locus")
region = gene_locus.find("Gene-commentary/Gene-commentary_seqs/Seq-loc/Seq-loc_int/Seq-interval")
start = int(region.find("Seq-interval_from").text) + 1
end = int(region.find("Seq-interval_to").text) + 1
gi_id = region.find("Seq-interval_id/Seq-id/Seq-id_gi").text
strand = region.find("Seq-interval_strand/Na-strand").get("value")
print(start, end, gi_id, strand)

12884422 12887201 568815579 minus


In [72]:
handle = Entrez.efetch(db="gene", id=int(dis_1_gen_0), rettype="gb", retmode="text")
# print(handle.readline().strip())

In [73]:
myline = handle.readline()
while myline:
    print(myline)
    myline = handle.readline()
handle.close()



1. KLF1

Official Symbol: KLF1 and Name: KLF transcription factor 1 [Homo sapiens (human)]

Other Aliases: EKLF, EKLF/KLF1

Other Designations: Krueppel-like factor 1; Kruppel like factor 1; erythroid Kruppel-like factor; erythroid krueppel-like transcription factor; erythroid-specific transcription factor EKLF

Chromosome: 19; Location: 19p13.13

Annotation: Chromosome 19 NC_000019.10 (12884422..12887201, complement)

MIM: 600599

ID: 10661



